In [1]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 73.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
!pip install Penman unidecode amrlib==0.5.1

  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 12.8 MB/s eta 0:00:00
  Created wheel for smatch: filename=smatch-1.0.4-py3-none-any.whl size=24054 sha256=953bf20a543d266ca054eeb660a14ccba37740c9f64490b1b8e14dcf4cf925f1
  Stored in directory: /root/.cache/pip/wheels/2a/5e/2d/61b88bc74b337fbf3e998b9aa6b43bec72227e18a84a8335e8
Successfully built smatch


In [3]:
# !git clone https://github.com/IBM/graph_ensemble_learning.git

In [4]:
# !cp -r graph_ensemble_learning/.* ./

In [5]:
!rm -r *

In [6]:
!cp -r /kaggle/input/amr-data/graph_ensemble_learning_new/* ./

In [7]:
!cat amr_parsing/t5/cli/train.py

import argparse
import os
import random
import warnings
from pathlib import Path

import penman
import torch
from amrlib.evaluate.smatch_enhanced import get_entries, compute_smatch
from amrlib.utils.logging import silence_penman
from penman.models.noop import NoOpModel
from torch import optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from amr_utils.datasets.dataset import AMRPenman, AMRPenmanMultiTasks, add_prefix, CombineData, AMR_GENERATION
from .inference import Inference
from ..models.lg import LG

import gc

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
warnings.simplefilter('ignore')


def save(optimizer, model, path):
    Path(path).mkdir(exist_ok=True)
    with open(os.path.join(path, f'multitask.model'), 'wb+') as f:
        if torch.cuda.device_count() > 1:
            torch.save({
                'model_state_dict': model.model.module.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, f)
 

In [8]:
!ls

__notebook__.ipynb  amr_parsing  amr_utils  ensemble


In [9]:
# !rm -r DATA_AMR_temp/
# !rm -r LDC2017T10/

# !mkdir -p DATA_AMR_temp/training/
# !mkdir -p DATA_AMR_temp/dev/
# !mkdir -p DATA_AMR_temp/test/

# # !cp -r /content/drive/MyDrive/AMR_data/amr-bank-struct-v1.6.txt ./DATA_AMR_temp/training/
# # !cp -r /content/drive/MyDrive/AMR_data/amr-bank-struct-v1.6.txt ./DATA_AMR_temp/dev/
# # !cp -r /content/drive/MyDrive/AMR_data/amr-bank-struct-v1.6.txt ./DATA_AMR_temp/test/

# # !cp -r /kaggle/input/amr-data/vn_data/vn_data/train_amr_v1_en_form.txt ./DATA_AMR_temp/training
# # !cp -r /kaggle/input/amr-data/vn_data/vn_data/train_amr_v1_en_form.txt ./DATA_AMR_temp/dev
# # !cp -r /kaggle/input/amr-data/vn_data/vn_data/train_amr_v1_en_form.txt ./DATA_AMR_temp/test

# !cp -r /kaggle/input/amr-data/train_amr_v3_en_form.txt ./DATA_AMR_temp/training
# !cp -r /kaggle/input/amr-data/train_amr_v3_en_form.txt ./DATA_AMR_temp/dev
# !cp -r /kaggle/input/amr-data/train_amr_v3_en_form.txt ./DATA_AMR_temp/test

In [10]:
import re

In [11]:
# def convert_data(file_name):
#   with open(file_name, "r") as f:
#     content = f.read()
#     f.close()

#   content = re.sub(r"#::", "# ::", content)

#   content = re.sub(r"\n\n", "\n", content)
    
#   if "v3" in file_name: content = re.sub(r"# ::snt", "\n# ::snt", content)

#   with open(file_name, "w") as f:
#     f.write(content)
#     f.close()

# convert_data("./DATA_AMR_temp/training/train_amr_v3_en_form.txt")
# convert_data("./DATA_AMR_temp/dev/train_amr_v3_en_form.txt")
# convert_data("./DATA_AMR_temp/test/train_amr_v3_en_form.txt")

In [12]:
# ! python -u -m amr_utils.preprocess.preprocess_amr -i ./DATA_AMR_temp/ -o LDC2017T10/preprocessed_data/

In [13]:
!mkdir -p t5_amr

In [14]:
# !cat ./LDC2017T10/preprocessed_data/train.txt.features

In [15]:
!cp -r /kaggle/input/amr-data/train_v4.jsonl ./
!cp -r /kaggle/input/amr-data/test_v4.txt ./

In [16]:
!ls

__notebook__.ipynb  amr_utils  t5_amr	    train_v4.jsonl
amr_parsing	    ensemble   test_v4.txt


In [17]:
# script = """
# python -u -m amr_parsing.t5.cli.train --train "./LDC2017T10/preprocessed_data/train.txt.features.nowiki" \
#     --validation ./LDC2017T10/preprocessed_data/dev.txt.features.nowiki \
#     --report_test ./LDC2017T10/preprocessed_data/test.txt.features.nowiki \
#     --max_source_length 512 --max_target_length 512 --batch 8 -e 30 -m vinai/bartpho-word-base \
#     --model_type bart --output ./t5_amr/ --data_type "amrdata" \
#     --task_type "text2amr" --val_from_epoch 2
# """

# script = """
# python -u -m amr_parsing.t5.cli.train --train "./LDC2017T10/preprocessed_data/train.txt.features" \
#     --validation ./LDC2017T10/preprocessed_data/dev.txt.features \
#     --report_test ./LDC2017T10/preprocessed_data/test.txt.features \
#     --max_source_length 512 --max_target_length 512 --batch 8 -e 40 -m vinai/bartpho-word-base \
#     --model_type bart --output ./t5_amr/ --data_type "amrdata" \
#     --task_type "text2amr" --val_from_epoch 20 --lr 5e-5
# """


script = """
python -u -m amr_parsing.t5.cli.train --train "/kaggle/working/train_v4.jsonl" \
    --validation /kaggle/working/train_v4.jsonl \
    --report_test /kaggle/working/train_v4.jsonl \
    --max_source_length 512 --max_target_length 512 --batch 2 -e 40 -m vinai/bartpho-word-base \
    --model_type bart --output ./t5_amr/ --data_type "amrdata" \
    --task_type "text2amr" --val_from_epoch 2 --lr 5e-5 --do_val "no"
"""

with open("a.bash" , "w") as f:
  f.write(script)
  f.close()

!cat a.bash


python -u -m amr_parsing.t5.cli.train --train "/kaggle/working/train_v4.jsonl"     --validation /kaggle/working/train_v4.jsonl     --report_test /kaggle/working/train_v4.jsonl     --max_source_length 512 --max_target_length 512 --batch 2 -e 40 -m vinai/bartpho-word-base     --model_type bart --output ./t5_amr/ --data_type "amrdata"     --task_type "text2amr" --val_from_epoch 2 --lr 5e-5 --do_val "no"


In [18]:
!bash a.bash

config.json: 100%|█████████████████████████████| 898/898 [00:00<00:00, 4.96MB/s]
vocab.txt: 100%|█████████████████████████████| 895k/895k [00:00<00:00, 12.7MB/s]
bpe.codes: 100%|███████████████████████████| 1.14M/1.14M [00:00<00:00, 13.5MB/s]
tokenizer.json: 100%|██████████████████████| 3.13M/3.13M [00:00<00:00, 26.8MB/s]
You are using a model of type mbart to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.
pytorch_model.bin: 100%|█████████████████████| 600M/600M [00:15<00:00, 39.6MB/s]
/kaggle/working/train_v4.jsonl
Number of file: 1
Data size: 101
Combine datasets size: 101
Epoch: 1; Loss: 2.43172; Moving Loss: 4.05904;: 100%|█| 51/51 [00:04<00:00, 11.0
/kaggle/working/train_v4.jsonl
Number of file: 1
Data size: 101
Combine datasets size: 101
Epoch: 2; Loss: 2.16128; Moving Loss: 2.11990;: 100%|█| 51/51 [00:03<00:00, 13.8
/kaggle/working/train_v4.jsonl
Number of file: 1
Data size: 101
Combine datasets size: 101
Epoch: 3; 

In [19]:
import torch
import penman
import os
from amr_parsing.t5.models.lg import LG
from amr_parsing.t5.cli.inference import Inference
from amr_utils.datasets.dataset import AMRPenman, AMRPenmanMultiTasks, add_prefix, CombineData, AMR_GENERATION

In [20]:
def format_amr(amr_string):
    indent = 6
    result = ''
    level = 0

    i = 0
    while i < len(amr_string):
        char = amr_string[i]
        if char == '(':
            if i > 0 and amr_string[i - 1] != ' ':
                result += ' '
            result += char
            level += 1
            # result += '\n' + ' ' * (level * indent)
        elif char == ')':
            # if amr_string[i-1] == ")":
            #     result += char
            # else:
              level -= 1
              result +=  char
        elif char == ':':
            result += '\n' + ' ' * (level * indent) + char
        else:
            result += char
        i += 1

    return result.strip()

In [21]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device("cpu")
print(device)


cuda


In [22]:
net = LG("vinai/bartpho-word-base",
             max_source_length=512,
             max_target_length=512,
             model_type="bart").to(device)

You are using a model of type mbart to instantiate a model of type bart. This is not supported for all configurations of models and can yield errors.
/opt/conda/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [23]:
net.model.load_state_dict(torch.load("./t5_amr/multitask.model", map_location=torch.device(device))["model_state_dict"])
net.eval()

LG(
  (model): BartForConditionalGeneration(
    (model): BartModel(
      (shared): Embedding(64001, 768, padding_idx=1)
      (encoder): BartEncoder(
        (embed_tokens): BartScaledWordEmbedding(64001, 768, padding_idx=1)
        (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
        (layers): ModuleList(
          (0-5): 6 x BartEncoderLayer(
            (self_attn): BartSdpaAttention(
              (k_proj): Linear(in_features=768, out_features=768, bias=True)
              (v_proj): Linear(in_features=768, out_features=768, bias=True)
              (q_proj): Linear(in_features=768, out_features=768, bias=True)
              (out_proj): Linear(in_features=768, out_features=768, bias=True)
            )
            (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (activation_fn): GELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=7

In [24]:
import re

In [25]:
test_path = "./test_v4.txt"

In [26]:
with open(test_path, "r") as f:
  test_data = f.read()
  # pattern = r"^#::snt.*"

  # Extract lines using regex
  # test_data = re.findall(pattern, test_data, re.MULTILINE)
  test_data = test_data.split("\n")
print(test_data)

['- đúng thế , cáo nói .', '- nhưng cậu sẽ khóc ! ông hoàng nhỏ nói .', '- thế thì cậu chẳng được gì cả !', '- được chứ , cáo nói , là do cái màu vàng của lúa mì ấy .', 'rồi nó nói thêm :', '- hãy trở lại thăm những đoá hồng đi .', 'cậu sẽ thấy đoá hoa của cậu là duy nhất trên đời .', 'rồi cậu hãy lại đây từ biệt tớ , tớ sẽ làm quà cho cậu một điều bí mật .', 'ông hoàng nhỏ đi thăm lại những đoá hoa hồng .', '- các cô chẳng giống chút nào với đoá hồng của tôi , các cô chưa là gì cả , em bảo các bông hồng .', 'chưa ai cảm hoá các cô , các cô cũng chưa cảm hoá ai .', 'các cô giống như con cáo của tôi trước kia .', 'nó chỉ là một con cáo giống như trăm nghìn con cáo .', 'và các bông hồng hết sức lúng túng .', '- các cô đẹp , nhưng các cô trống rỗng , em nói với họ .', 'người ta không thể chết vì các cô được .', 'phải , đoá hồng của tôi , một người qua đường tầm thường tưởng là nàng giống các cô .', 'nhưng đối với tôi thì nàng quan trọng hơn tất cả các cô , bởi vì chính là nàng mà tay tôi 

In [27]:
from tqdm import autonotebook
pbar = autonotebook.tqdm(range(len(test_data)))

  0%|          | 0/151 [00:00<?, ?it/s]

In [28]:
import re

In [29]:
gen_fname = "result_v4_test.txt"
result = ""
for i, text in enumerate(test_data):
  pbar.update(1)
  if text == "":
    continue
  # add_prefix(sentence, AMR_GENERATION)
  predict_amrs = net.generate(add_prefix([text], AMR_GENERATION), max_decoder_batch_size=256,)
  predict_amrs = re.sub(r"\( ", "(", predict_amrs[0])
  predict_amrs = re.sub(r" \)", ")", predict_amrs)
  predict_amrs = format_amr(predict_amrs)
  # print(predict_amrs)
  # break
  if predict_amrs is not None:
    result = result + f"#::id {i}\n#::snt {text}\n\n" + predict_amrs + "\n\n"


In [30]:
with open(gen_fname, "w") as f:
  f.write(result)
  f.close()